In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_path = 'rl4lm_ans2que_exp_failed/rl4lm_experiment/model/'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)


In [13]:
from transformers import AutoTokenizer


model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
import pandas as pd
from pathlib import Path
import os

In [4]:
dataset_path = os.path.join(Path.home(), "narrative_qa",'val.tsv')
narrative_qa_df = pd.read_csv(dataset_path, sep='\t',names=['question','answers'], header=None)

In [ ]:
narrative_qa_df.head(9)

In [6]:
for i in range(2,narrative_qa_df.shape[0]-2, 2): 
    if narrative_qa_df['question'][i] == narrative_qa_df['question'][i+1]:
        continue
    else:
        print(f'False, {i}')
        

In [7]:


def generate_question(answer):
    model.eval() 
    input_ids = tokenizer.encode(answer, return_tensors="pt").to(device)
    outputs = model.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [8]:
import random

random_index = random.randint(0, len(narrative_qa_df) - 1)
selected_answer = narrative_qa_df.iloc[random_index]['answers']+'s'


In [ ]:
selected_answer

In [ ]:
generated_question = generate_question(selected_answer)
print(f"Answer: {selected_answer}")
print(f"Generated Question: {generated_question}")
print(f"Actual Question: ", narrative_qa_df.iloc[random_index]['question'].split('\\n')[0])
context_str = narrative_qa_df.iloc[random_index]['question'].split('\\n')[1]
print("Context: ", context_str)

In [ ]:
import os
import openai

openai.api_key = 'your api key'


chat_completion = openai.chat.completions.create(
    messages=[
        {"role": "user", "content": f"Generate the possible question of this answer based on the context : Answer: {selected_answer}"}
    ],
    model="gpt-4-turbo",
    max_tokens=100,
)

print("GPT 4-turbo: ",chat_completion.choices[0].message.content)
print("Actual Question: ", narrative_qa_df.iloc[random_index]['question'].split('\\n')[0])


In [ ]:
narrative_qa_df.iloc[random_index]['question'].split('\\n')[0]
